In [ ]:
## import modules
import pandas as pd
import numpy as np
import datetime
import scipy as sp
from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
import os
from sklearn.metrics import roc_auc_score
import math

In [ ]:
## import data 
all_data = pd.read_csv('data/hPRB1-conc-data_1-1034_serum-and-saliva.csv')
all_data

In [ ]:
## Extract features for use
feature_data = all_data.loc[:, ['Sample_ID', 'Serum_hPRB1-normalised']]
feature_data = feature_data.set_index('Sample_ID')
feature_data

In [ ]:
## Rename data column-name
feature_data = feature_data.rename(columns={"Serum_hPRB1-normalised":"Serum_hPRB1_normalised"})
feature_data

In [ ]:
## Scale values by 1e-6
feature_data['Serum_hPRB1_normalised_change_unit'] = feature_data['Serum_hPRB1_normalised']/1e-6

In [ ]:
# import tbp list
tbp_dfp_list = pd.read_csv('data/Disease_analysing-list.csv',encoding='shift-jis')
tbp_dfp_list

In [ ]:
## set features list and error list
feature_list = ['Serum_hPRB1_normalised_change_unit']
err_list = []

coeff_sum = None

# Loop of the disease to be predicted
for tbp_t in range(len(tbp_dfp_list)):
    cur_dis_type = tbp_dfp_list.disease_class[tbp_t]
    cur_dis_name = tbp_dfp_list.disease_name[tbp_t]
    
    print(tbp_t, ':', cur_dis_type, '__', cur_dis_name, ' --- ', datetime.datetime.today())
    
    cur_folder_data_fp = ''.join(['data/Disease_binary-data/',
                                tbp_dfp_list.label[tbp_t]])
    
    cur_folder_data_fp = cur_folder_data_fp.replace('Single', 'single')
    
    cur_tbp_data = pd.read_csv(cur_folder_data_fp, 
                                index_col='Sample_ID',encoding='Shift-JIS')
    
    cur_disease = cur_tbp_data.columns[0]
    
    ## data merge
    merge_data = pd.concat([cur_tbp_data, feature_data], axis=1)
    
    #for fea in [feature_list[0]]:
    for fea in feature_list:
        print(fea, '---', datetime.datetime.today())

        analyse_data = merge_data.loc[:, [cur_disease, fea]].dropna(how='any',axis=0)
        analyse_data.columns.values[0] = 'proxy'
        
        ## logistic modeling
        mod_glm = smf.glm(formula = "".join(["proxy ~ ", fea]),
                            data = analyse_data,
                            family = sm.families.Binomial()).fit()
            
        y_pred = mod_glm.predict(analyse_data.loc[:,[fea]])
        
        coeff_df = pd.DataFrame(data=[cur_dis_type, cur_dis_name, np.exp(mod_glm.params[1]), mod_glm.pvalues[1],
                                    -1*math.log10(mod_glm.pvalues[1]),
                                    roc_auc_score(y_true=analyse_data.loc[:,['proxy']], y_score=mod_glm.predict(analyse_data.loc[:,[fea]]))],
                                index =  ["Analysis_type", "Disease_name", "OddsRatio_1unit=1e-6", "pvalue", "-log10pvalue", "AUC"])
        coeff_df = coeff_df.T
        
        if coeff_sum is None:
            coeff_sum = coeff_df
        else:
            coeff_sum = pd.concat([coeff_sum, coeff_df], axis = 0)

In [ ]:
coeff_sum.to_csv('results/Disease-association-with-serum-hPRB1_odds-results.csv', encoding='shift-jis', index=False)